In [105]:
# import spacy
# import textacy


# print('loading en_coref_md...')
# nlp = spacy.load('en_coref_md')
# print('...done')



# for now assuming all names are unique identifiers
class Person:
    statements = []
#     resolved_refs = []

    def __init__(self, name, refs=[]):
        self.name = name
#         self.gender = gender
        self.refs = refs
#         self.user = user




# UPGRADE AT SOME POINT TO EXTRACT GENDER, ACCOUNT FOR CLUSTERS WITHOUT NAMES
# UPGRADE TO INCLUDE I, USER

# assumes names are unique identifiers
# assumes misspellings are diff people

# MEMORYLESS FOR NOW; each change to text means a whole new model
# Set extensions later, for keeping track of which tokens are what
class Model:
    raw = None
    doc = None
    people = []
    resolved = None

    def __init__(self, text):
        raw = text
        preprocessed = textacy.preprocess.normalize_whitespace(text)
        preprocessed = textacy.preprocess.preprocess_text(preprocessed, fix_unicode=True, no_contractions=True, no_accents=True)
        self.doc = nlp(preprocessed)
        self.extract_people()
        self.resolved = self.get_resolved()

    def get_person_by_name(self, name):
        for person in self.people:
            if person.name == name:
                return person
        return None

    def extract_people(self, doc=None):
        if doc == None:
            doc = self.doc
        namedrops = [ent for ent in doc.ents if ent.label_ == 'PERSON']
        names = set([namedrop.text for namedrop in namedrops])
        
        # for clusters that include namedrops
        for cluster in doc._.coref_clusters:
            name = None

            for mention in cluster.mentions:
                mention_text = mention.root.text
                if mention_text in names:
                    name = mention_text

            if name != None:
                person = self.get_person_by_name(name)
                if person == None:
                    person = Person(name, refs=cluster.mentions)
                    self.people += [person]
                else:
                    person.refs = list(set(person.refs + cluster.mentions))

            # for named entities without clusters (single mentions)
            for name_mention in namedrops:
                person = self.get_person_by_name(name_mention.text)
                if person == None:
                    person = Person(name_mention.text, refs=[name_mention])
                    self.people += [person]
                else:
                    person.refs = list(set(person.refs + cluster.mentions))
                    
#     def get_resolved_text(self, doc=None):
#         if doc == None:
#             doc = self.doc
#         tokens = [token.text for token in doc]
#         for person in self.people:
#             for ref in person.refs:

#                 # determine resolved value
#                 resolved_token = person.name
#                 if ref.root.pos_ == 'ADJ':
#                     resolved_token += '\'s'

#                 # set first token to resolved value
#                 tokens[ref.start] = resolved_token

#                 # set extra tokens in mention to blank
#                 for i in range(ref.start+1, ref.end):
#                     tokens[i] = ''
#         return ' '.join([token for token in tokens if token != ''])

#     def resolve(self):
#         self.resolved_text = self.get_resolved_text()
#         self.resolved_doc = nlp(self.resolved_text)

#         offset = 0;
#         for person in self.people:
#             for ref in person.refs:
#                 resolved_ref = self.resolved_doc[ref.start-offset:ref.start-offset+1]
#                 person.resolved_refs += [resolved_ref]

#                 # increase offset for each multi-word ref
#                 words_in_ref = (ref.end - ref.start)
#                 offset += words_in_ref - 1
#     def resolve(self):
                
#         # create resolved text
#         chars = list(self.doc.text)
#         chars_inserted_before = 0
#         for person in self.people:
#             for ref in person.refs:
                
#                 # determine resolved value
#                 resolved = person.name
#                 if ref.root.pos_ == 'ADJ':
#                     resolved += '\'s'
#                 if ref.text_with_ws[-1] = ' ':
#                     resolved += ' '


#                 # set appropriate chars to resolved value
#                 new_start = ref.start_char + chars_inserted_before
#                 new_end = ref.end_char + chars_inserted_before
#                 print(chars_inserted_before, new_start, new_end, chars[new_start:new_end], list(resolved))
#                 chars[new_start:new_end] = list(resolved)
#                 chars_inserted_before += len(resolved) - len(ref.text)
                
#         self.resolved_doc = nlp(''.join(chars))
        
#         # create list with original text but resolved token index
#         merged_copy = copy(self.doc)
#         for person in self.people:
#             for ref in person.refs:
#                 merged_copy.char_span(ref.start_char, ref.end_char).merge()
        
#         # use merged_copy to translate refs to resolved_refs
#         for person in self.people:
#             for ref in person.refs:
#                 resolved_i = merged_copy.char_span(ref.char_start, ref.char_end)
#                 resolved_ref = self.resolved_doc[resolved_i]
#                 person.resolved_refs = list(set(person.resolved_refs + resolved_ref))
    
    def get_resolved(self, doc=None):
        if doc == None:
            doc = self.doc
            
        resolved_text = [token.text_with_ws for token in self.doc]
        for person in self.people:
            for ref in person.refs:
            
                # determine resolved value
                resolved_value = person.name
                if ref.root.pos_ == 'ADJ':
                    resolved_value += '\'s'
                if ref.text_with_ws[-1] == ' ':
                    resolved_value += ' '

                # set first token to value, remaining tokens to ''
                resolved_text[ref.start] = resolved_value
                for i in range(ref.start+1, ref.end):
                    resolved_text[i] = ''   
        return ''.join(resolved_text)
    
#     def update_people_statements(self, doc):
#         res = nlp(self.resolve_people(doc))
    
#         for person in model.people:
#             statements = []
#             for ref in person.refs:
#                 head = ref.root.head
#                 if head.pos_ == 'VERB':
#                     for statement in textacy.extract.semistructured_statements(res, person.name, head.lemma_):
#                         statements += [statement]
#             person.statements = list(set(statements))


In [106]:
text = "Sheila was run over by a truck. She herself didn't see that coming. I told her she should take care of herself, but I know she'll just go and do her thing regardless of what I say. What a conundrum! This makes me wish I had never signed up to be friends with her, although I do love the girl."

In [130]:
doc = ('we need to go home')
doc[0:0]

''

In [114]:
model = Model(text)
model.resolved

TypeError: 'NoneType' object is not iterable

In [129]:
for cluster in nlp("Today is the 10th anniversary of the day that Katy adopted me. It’s also the 4th anniversary of the day she left me. I’ve been wanting to pay tribute to her, but this week has been so frustrating with ups and downs and maybes. It took an episode of Life Below Zero to draw it out of me. For as long as I can remember, I wanted to be a mama. I got married young but wanted to wait until college was over and I was established in my career before we started a family. I began to have a nagging feeling right away, that I’d never have a baby. I thought I was going to die young. I prayed and prayed a selfish prayer that God would just let me have that experience before he took me. As you may or may not know, my marriage ended before children came along. I had one unplanned pregnancy after that, but it ended in disaster after only a few weeks. As those of you know who have had a baby in your womb but never got to hold her in your arms, there is a magical transformation that takes place that makes you a member of a new club. Tonight I am frustrated again. I’ve learned that the test that is required to determine my “disability status” will cost $800, and is not covered by my insurance. As I said, the ups and downs and maybes are taking their toll this week. When we got home, I warned Drew that I’m emotional and please don’t take it personally, and then plopped into my chair. He brought me dinner then put on Life Below Zero, where one of the men featured was calling to his dog and she wasn’t moving. I already had tears in my eyes and a lump in my throat, anticipating his loss. But as he reached into her house to touch her, she looked up at him and revealed a litter of puppies. I’ve been crying ever since. When Katy came along, she had big paws to fill. Magnum, my first dog, had been an absolute angel. He had the most gentle temperament, but he was also a fierce protector. It was years after losing him before I could think of getting another dog. But the time came, that I was ready for another dog. I was going to be moving into a house by myself and wanted a medium sized dog as an early warning alarm if nothing else. The woman who was trying to find a home for her told me that she’d been abandoned at a trailer park. She was tied to a trailer where she was fed scraps by various park residents. That’s why she was so fat. The woman brought Katy to my apartment to meet me. I sat down on the floor across the room, and said, “Hi, Katy!” She walked straight to me and laid her head in my lap. She cuddled me like it was a familiar place for her. I think we both knew she was home. Katy was ravenous that first week or so. She’d ransack the garbage at night and she’d inhale her food. I’d accidentally left out a log of premium wet dog food. She had eaten half of it by the time I realized it. I couldn’t understand why she was so hungry, but I understood why she was so fat now. Or so I thought. One day I came home from work and found her laying in the kitchen with her head propped against a cabinet. I suddenly suspected that she wasn’t just fat, she might be pregnant! I got down on the floor with her and put my hands on her belly. I could feel the babies moving around!")._.coref_clusters:
    for mention in cluster.mentions:
        print(mention.text, '-', [token.text for token in mention.root.subtree])

the 10th anniversary of the day that Katy adopted me - ['the', '10th', 'anniversary', 'of', 'the', 'day', 'that', 'Katy', 'adopted', 'me']
It - ['It']
Katy - ['Katy']
she - ['she']
her - ['her']
this week - ['this', 'week']
It - ['It']
an episode of Life Below Zero - ['an', 'episode', 'of', 'Life', 'Below', 'Zero']
it - ['it']
God - ['God']
he - ['he']
one unplanned pregnancy - ['one', 'unplanned', 'pregnancy']
it - ['it']
a baby - ['a', 'baby']
her - ['her']
the ups and downs and maybes - ['the', 'ups', 'and', 'downs', 'and', 'maybes']
their - ['their']
He - ['He']
his - ['his']
his - ['his']
he - ['he']
him - ['him']
He - ['He']
he - ['he']
him - ['him']
myself - ['myself']
his dog - ['his', 'dog']
she - ['she']
her - ['her']
her - ['her']
she - ['she']
Katy - ['Katy']
she - ['she']
The woman who was trying to find a home for her - ['The', 'woman', 'who', 'was', 'trying', 'to', 'find', 'a', 'home', 'for', 'her']
her - ['her']
she - ['she']
She - ['She']
she - ['she']
she - ['she']
Th

In [98]:
for token in model.doc:
    print('-' + token.text_with_ws + '-')

-Sheila -
-was -
-run -
-over -
-by -
-a -
-truck-
-. -
-She -
-herself -
-did -
-not -
-see -
-that -
-coming-
-. -
-I -
-told -
-her -
-she -
-should -
-take -
-care -
-of -
-herself-
-, -
-but -
-I -
-know -
-she -
-will -
-just -
-go -
-and -
-do -
-her -
-thing -
-regardless -
-of -
-what -
-I -
-say-
-. -
-What -
-a -
-conundrum-
-! -
-This -
-makes -
-me -
-wish -
-I -
-had -
-never -
-signed -
-up -
-to -
-be -
-friends -
-with -
-her-
-, -
-although -
-I -
-do -
-love -
-the -
-girl-
-.-
-Sheila -
-was -
-run -
-over -
-by -
-a -
-truck-
-. -
-She -
-herself -
-did -
-not -
-see -
-that -
-coming-
-. -
-I -
-told -
-her -
-she -
-should -
-take -
-care -
-of -
-herself-
-, -
-but -
-I -
-know -
-she -
-will -
-just -
-go -
-and -
-do -
-her -
-thing -
-regardless -
-of -
-what -
-I -
-say-
-. -
-What -
-a -
-conundrum-
-! -
-This -
-makes -
-me -
-wish -
-I -
-had -
-never -
-signed -
-up -
-to -
-be -
-friends -
-with -
-her-
-, -
-although -
-I -
-do -
-love -
-the -
-girl-


-know -
-she -
-will -
-just -
-go -
-and -
-do -
-her -
-thing -
-regardless -
-of -
-what -
-I -
-say-
-. -
-What -
-a -
-conundrum-
-! -
-This -
-makes -
-me -
-wish -
-I -
-had -
-never -
-signed -
-up -
-to -
-be -
-friends -
-with -
-her-
-, -
-although -
-I -
-do -
-love -
-the -
-girl-
-.-
-Sheila -
-was -
-run -
-over -
-by -
-a -
-truck-
-. -
-She -
-herself -
-did -
-not -
-see -
-that -
-coming-
-. -
-I -
-told -
-her -
-she -
-should -
-take -
-care -
-of -
-herself-
-, -
-but -
-I -
-know -
-she -
-will -
-just -
-go -
-and -
-do -
-her -
-thing -
-regardless -
-of -
-what -
-I -
-say-
-. -
-What -
-a -
-conundrum-
-! -
-This -
-makes -
-me -
-wish -
-I -
-had -
-never -
-signed -
-up -
-to -
-be -
-friends -
-with -
-her-
-, -
-although -
-I -
-do -
-love -
-the -
-girl-
-.-
-Sheila -
-was -
-run -
-over -
-by -
-a -
-truck-
-. -
-She -
-herself -
-did -
-not -
-see -
-that -
-coming-
-. -
-I -
-told -
-her -
-she -
-should -
-take -
-care -
-of -
-herself-
-, -
-but -
-

-her-
-, -
-although -
-I -
-do -
-love -
-the -
-girl-
-.-
-Sheila -
-was -
-run -
-over -
-by -
-a -
-truck-
-. -
-She -
-herself -
-did -
-not -
-see -
-that -
-coming-
-. -
-I -
-told -
-her -
-she -
-should -
-take -
-care -
-of -
-herself-
-, -
-but -
-I -
-know -
-she -
-will -
-just -
-go -
-and -
-do -
-her -
-thing -
-regardless -
-of -
-what -
-I -
-say-
-. -
-What -
-a -
-conundrum-
-! -
-This -
-makes -
-me -
-wish -
-I -
-had -
-never -
-signed -
-up -
-to -
-be -
-friends -
-with -
-her-
-, -
-although -
-I -
-do -
-love -
-the -
-girl-
-.-
-Sheila -
-was -
-run -
-over -
-by -
-a -
-truck-
-. -
-She -
-herself -
-did -
-not -
-see -
-that -
-coming-
-. -
-I -
-told -
-her -
-she -
-should -
-take -
-care -
-of -
-herself-
-, -
-but -
-I -
-know -
-she -
-will -
-just -
-go -
-and -
-do -
-her -
-thing -
-regardless -
-of -
-what -
-I -
-say-
-. -
-What -
-a -
-conundrum-
-! -
-This -
-makes -
-me -
-wish -
-I -
-had -
-never -
-signed -
-up -
-to -
-be -
-friends -
-wit

NameError: name 'doc' is not defined

In [40]:
len(model.people)

1

In [41]:
[person.refs for person in model.people]

[[She herself, the girl, her, she, her, she, her, herself, She, Sheila]]

In [42]:
for person in model.people:
    for ref, res in zip(person.refs, person.resolved_refs):
        print(ref.start, ref, res.start, res)
#     print(person.name, person.refs, person.resolved_refs)

8 She herself 8 Sheila
66 the girl 65 love
60 her 58 friends
19 she 17 Sheila
18 her 16 told
29 she 27 know
35 her 33 do
24 herself 22 of
8 She 6 truck
0 Sheila 66 Sheila


In [13]:
print(len([token for token in model.doc]), len([token for token in model.resolved_doc]))

69 68


In [14]:
text = 'November was a trying month… on the 7th Dante had a major accident. 5 minutes before school and he and some friends are climbing the fence, I tell him it’s not a good idea and to get down. I turn back to talk to Jodi (on of my best mom friend’s at the school) and Dante comes to me screaming with his hand full of blood. I run him into my classroom and get him to the sink, as I turn on the water to clean the area the flap of his thumb lifts away and I see the bone. Shit. This isn’t something I can fix here, I grab my first aid kit and wrap it like crazy because it’s bleeding like crazy. I phone James and tell him to get to the ER as Dante is screaming and freaking out in the background as I’m trying to usher him back to the car as he’s bleeding like a stuffed pig. Unfortunately in the ER I learned that my child doesn’t take to freezing, an hour of gel freezing and he still felt the 2 needles as they went in, 15 minutes later and he felt the last 2 stitches of 8. He needed more because his finger still had gaps, the doctor didn’t want to cause him anymore pain so he glued them. It was an intense and deep gash that spiraled all the way up his thumb. I was trying to stay strong for him but I did break down as he screamed and cried, I was left to emotionally drained that day. James was able to take the remainder of the day off and stay with him. He missed 2 more days of school and then had an extra long weekend due to the holiday and the pro day but for 2 weeks he couldn’t write (of course it was his right hand.) 3 doctor visits later and he finally got them out full last week, the first visit the doctor wanted them in longer because of the severity. 2nd time he could only get 6 out because the glue had gotten on the last 2 stitches and he didn’t want to have to dig them out so we had to soak and dissolve the glue for 3 days. 3rd time the last 2 came out.  Even now he’s slowly regaining his writing skills as there was some nerve damage.'

In [9]:
adoc = nlp(text)
# for cluster in adoc._.coref_clusters:
#     for mention in cluster.mentions:
#         mention.merge()
        
for cluster in adoc._.coref_clusters:
    for mention in cluster.mentions:
        print(cluster)

the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
school: [school, the school]
school: [school, the school]
my first aid kit: [my first aid kit, it, it]
my first aid kit: [my first aid kit, it, it]
my first aid kit: [my first aid kit, it, it]


IndexError: [E037] Error calculating span: Can't find a token ending at character offset 1001.

In [10]:
adoc = nlp(text)
# for cluster in adoc._.coref_clusters:
#     for mention in cluster.mentions:
#         mention.merge()
        
for cluster in adoc._.coref_clusters:
    for mention in cluster.mentions:
        print(cluster)

the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
school: [school, the school]
school: [school, the school]
my first aid kit: [my first aid kit, it, it]
my first aid kit: [my first aid kit, it, it]
my first aid kit: [my first aid kit, it, it]
James: [James, him, him, he, he, he, He, his, him, he, his, him, he]
James: [James, him, him, he, he, he, He, his, him, he, his, him, he]
James: [James, him, him, he, he, he, He, his, him, he, his, him, he]
James: [James, him, him, he, he, he, He, his, him, he, his, him,